In [1]:
import findspark

findspark.init()

import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, transform

In [2]:
spark = SparkSession.builder.appName("BankAnalysis") \
    .config("spark.driver.host", "127.0.0.1") \
    .config("spark.driver.bindAddress", "127.0.0.1").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/08/12 05:02:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/12 05:06:46 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [5]:
bankText = spark.read.text("/user/seonggil/bank-full.csv").rdd.map(lambda r: r[0])

In [9]:
from collections import namedtuple

Bank = namedtuple("Bank", ["age", "job", "marital", "education", "balance"])


def parse_line(line):
    parts = line.split(";")
    return Bank(int(parts[0]),
                parts[1].replace("\"", ""),
                parts[2].replace("\"", ""),
                parts[3].replace("\"", ""),
                int(parts[5].replace("\"", "")))

In [10]:
bank = bankText.filter(lambda s: "age" not in s).map(parse_line)

In [11]:
# RDD를 DataFrame으로 변환하고 임시 테이블 생성

bank_df = spark.createDataFrame(bank)
bank_df.createOrReplaceTempView("bank")

In [12]:
spark.sql("SELECT age, COUNT(1) FROM bank GROUP BY age ORDER BY age").show()

+---+--------+
|age|count(1)|
+---+--------+
| 18|      12|
| 19|      35|
| 20|      50|
| 21|      77|
| 22|     123|
| 23|     191|
| 24|     282|
| 25|     457|
| 26|     696|
| 27|     776|
| 28|     866|
| 29|     950|
| 30|    1312|
| 31|    1518|
| 32|    1587|
| 33|    1465|
| 34|    1446|
| 35|    1429|
| 36|    1363|
| 37|    1301|
+---+--------+
only showing top 20 rows



In [13]:
spark.sql("SELECT marital, COUNT(1) FROM bank GROUP BY marital").show()

+--------+--------+
| marital|count(1)|
+--------+--------+
|divorced|    4096|
| married|   21814|
|  single|    9843|
+--------+--------+



In [14]:
spark.sql("SELECT job, COUNT(1) FROM bank GROUP BY job ORDER BY COUNT(1) DESC").show()

+-------------+--------+
|          job|count(1)|
+-------------+--------+
|  blue-collar|    9732|
|   technician|    7597|
|       admin.|    5171|
|     services|    4154|
|      retired|    2264|
|self-employed|    1579|
| entrepreneur|    1487|
|   unemployed|    1303|
|    housemaid|    1240|
|      student|     938|
|      unknown|     288|
+-------------+--------+



In [3]:
spark.stop()